In [1]:
!{'pip install --quiet apache_beam'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

In [28]:
import apache_beam as beam

def SplitRow(row):
  return row.split(',')
with beam.Pipeline() as p:
  input_collection = (
      p
      | 'read from text file' >> beam.io.ReadFromText('drive/MyDrive/APACHE_BEAM/datasets/dept_data.txt')
      | 'split rows' >> beam.Map(SplitRow)
  )
  accounts_count = (
      input_collection
      | 'Get Accounts dept records' >> beam.Filter(lambda record: record[3]=="Accounts")
      | 'Pair Account record with 1' >> beam.Map(lambda record: ("Accounts: " + record[1], 1))
      | 'group and sum Account' >> beam.CombinePerKey(sum)
      # | 'Write results for account' >> beam.io.Write('data/Account')
  )

  hr_count = (
      input_collection
      | 'Get HR dept records' >> beam.Filter(lambda record: record[3]=="HR")
      | 'Pair HR record with 1' >> beam.Map(lambda record: ("HR: " + record[1], 1))
      | 'group and sum HR' >> beam.CombinePerKey(sum)
      # | 'Write results for account' >> beam.io.Write('data/Account')
  )

  final_count = (
      (accounts_count, hr_count)
      | beam.Flatten()
      | beam.io.WriteToText("drive/MyDrive/APACHE_BEAM/output/final_data.txt")
  )


In [29]:
!cat drive/MyDrive/APACHE_BEAM/output/final_data.txt-00000-of-00001

('Accounts: Marco', 31)
('Accounts: Rebekah', 31)
('Accounts: Itoe', 31)
('Accounts: Edouard', 31)
('Accounts: Kyle', 62)
('Accounts: Kumiko', 31)
('Accounts: Gaston', 31)
('Accounts: Ayumi', 30)
('HR: Beryl', 62)
('HR: Olga', 31)
('HR: Leslie', 31)
('HR: Mindy', 31)
('HR: Vicky', 31)
('HR: Richard', 31)
('HR: Kirk', 31)
('HR: Kaori', 31)
('HR: Oscar', 31)
